In [29]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 4: Camp Cleanup ---</h2><p>Space needs to be cleared before the last supplies can be unloaded from the ships, and so several Elves have been assigned the job of cleaning up sections of the camp. Every section has a unique <em>ID number</em>, and each Elf is assigned a range of section IDs.</p>
<p>However, as some of the Elves compare their section assignments with each other, they've noticed that many of the assignments <em>overlap</em>. To try to quickly find overlaps and reduce duplicated effort, the Elves pair up and make a <em>big list of the section assignments for each pair</em> (your puzzle input).</p>
<p>For example, consider the following list of section assignment pairs:</p>
<pre><code>2-4,6-8
2-3,4-5
5-7,7-9
2-8,3-7
6-6,4-6
2-6,4-8
</code></pre>
<p>For the first few pairs, this list means:</p>
<ul>
<li>Within the first pair of Elves, the first Elf was assigned sections <code>2-4</code> (sections <code>2</code>, <code>3</code>, and <code>4</code>), while the second Elf was assigned sections <code>6-8</code> (sections <code>6</code>, <code>7</code>, <code>8</code>).</li>
<li>The Elves in the second pair were each assigned two sections.</li>
<li>The Elves in the third pair were each assigned three sections: one got sections <code>5</code>, <code>6</code>, and <code>7</code>, while the other also got <code>7</code>, plus <code>8</code> and <code>9</code>.</li>
</ul>
<p>This example list uses single-digit section IDs to make it easier to draw; your actual list might contain larger numbers. Visually, these pairs of section assignments look like this:</p>
<pre><code>.234.....  2-4
.....678.  6-8

.23...... 2-3
...45.... 4-5

....567.. 5-7
......789 7-9

.2345678. 2-8
..34567.. 3-7

.....6... 6-6
...456... 4-6

.23456... 2-6
...45678. 4-8
</code></pre>

<p>Some of the pairs have noticed that one of their assignments <em>fully contains</em> the other. For example, <code>2-8</code> fully contains <code>3-7</code>, and <code>6-6</code> is fully contained by <code>4-6</code>. In pairs where one assignment fully contains the other, one Elf in the pair would be exclusively cleaning sections their partner will already be cleaning, so these seem like the most in need of reconsideration. In this example, there are <code><em>2</em></code> such pairs.</p>
<p><em>In how many assignment pairs does one range fully contain the other?</em></p>
</article>


In [30]:
def count_fully_contained_assignment_pairs(s: str) -> int:
    pairs = [
        [int(i) for i in re.split(r"\D+", l.strip())] for l in s.strip().splitlines()
    ]
    return sum(
        1
        for fr1, to1, fr2, to2 in pairs
        if fr1 <= fr2 <= to2 <= to1 or fr2 <= fr1 <= to1 <= to2
    )


count_fully_contained_assignment_pairs_tests = [
    {
        "name": "Example",
        "s": """
            2-4,6-8
            2-3,4-5
            5-7,7-9
            2-8,3-7
            6-6,4-6
            2-6,4-8
        """,
        "expected": 2,
    },
]


@test(tests=count_fully_contained_assignment_pairs_tests)
def count_fully_contained_assignment_pairs_test(s: str) -> int:
    return count_fully_contained_assignment_pairs(s)


Test Example passed, for count_fully_contained_assignment_pairs_test.
Success


In [31]:
with open("../input/day4.txt") as f:
    puzzle = f.read()

print(f"Part 1: {count_fully_contained_assignment_pairs(puzzle)}")

Part 1: 602


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>602</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>It seems like there is still quite a bit of duplicate work planned. Instead, the Elves would <span title="If you like this, you'll *love* axis-aligned bounding box intersection testing.">like</span> to know the number of pairs that <em>overlap at all</em>.</p>
<p>In the above example, the first two pairs (<code>2-4,6-8</code> and <code>2-3,4-5</code>) don't overlap, while the remaining four pairs (<code>5-7,7-9</code>, <code>2-8,3-7</code>, <code>6-6,4-6</code>, and <code>2-6,4-8</code>) do overlap:</p>
<ul>
<li><code>5-7,7-9</code> overlaps in a single section, <code>7</code>.</li>
<li><code>2-8,3-7</code> overlaps all of the sections <code>3</code> through <code>7</code>.</li>
<li><code>6-6,4-6</code> overlaps in a single section, <code>6</code>.</li>
<li><code>2-6,4-8</code> overlaps in sections <code>4</code>, <code>5</code>, and <code>6</code>.</li>
</ul>
<p>So, in this example, the number of overlapping assignment pairs is <code><em>4</em></code>.</p>
<p><em>In how many assignment pairs do the ranges overlap?</em></p>
</article>


In [ ]:
def count_overlapping_assignment_pairs(s: str) -> int:
    pairs = [
        [int(i) for i in re.split(r"\D+", l.strip())] for l in s.strip().splitlines()
    ]
    return sum(1 for fr1, to1, fr2, to2 in pairs if to2 >= fr1 and to1 >= fr2)


count_overlapping_assignment_pairs_tests = [
    {
        "name": "Example",
        "s": """
            2-4,6-8
            2-3,4-5
            5-7,7-9
            2-8,3-7
            6-6,4-6
            2-6,4-8
        """,
        "expected": 4,
    },
]


@test(tests=count_overlapping_assignment_pairs_tests)
def count_overlapping_assignment_pairs_test(s: str) -> int:
    return count_overlapping_assignment_pairs(s)


Test Example passed, for count_overlapping_assignment_pairs_test.
Success


<link href="style.css" rel="stylesheet"></link>


In [33]:
print(f"Part 2: {count_overlapping_assignment_pairs(puzzle)}")

Part 2: 891


<main>

<p>Your puzzle answer was <code>891</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
